https://wikidocs.net/256983

In [10]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [11]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Structured-Output-Chain")

LangSmith 추적을 시작합니다.
[프로젝트명]
Structured-Output-Chain


In [12]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

import time

from typing import List

### 1. 제공된 예제 코드 + 모델만 solar-pro 로 변경

In [13]:
start_time = time.time()

class Quiz(BaseModel):
    """4지선다형 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 4개의 선택지 입니다.")


llm = ChatUpstage(model="solar-pro", temperature=0.8)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 4지선다형 퀴즈를 한국어로 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 여러 개의 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "수능 지구과학2"})

# 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print()
    
end_time=time.time()
print(end_time-start_time)

1. What is the distance between the sun and the earth? (난이도: easy)
   1) 300 million kilometers
   2) 150 million kilometers
   3) 1 million kilometers
   4) 15 million kilometers

2. What is the largest constellation in the sky? (난이도: medium)
   1) Ursa Major
   2) Orion
   3) Pegasus
   4) Draco

3. What is the smallest planet in our solar system? (난이도: difficult)
   1) Mercury
   2) Venus
   3) Mars
   4) Pluto

9.722112655639648


### 2. O/X 로 변경

In [14]:
class Quiz(BaseModel):
    """OX 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 답은 O 또는 X 입니다.")

start_time = time.time()
llm = ChatUpstage(model="solar-pro", temperature=0.8)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 OX 퀴즈를 한국어로 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 여러 개의 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "수능 지구과학2"})

# 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):  
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print()

end_time = time.time()
print(end_time - start_time)

1. 수능 지구과학2에서, 만유인력의 법칙은 물리적 영향을 주는 원리를 설명합니다. (난이도: 보통)
   1) X
   2) X

2. 지구과학에서, 지구의 자전은 지구의 자전에 의해 발생합니다. (난이도: 어려움)
   1) O
   2) X

6.903207302093506


In [15]:
quiz

Quiz(question='지구과학에서, 지구의 자전은 지구의 자전에 의해 발생합니다.', level='어려움', options=['O', 'X'])

### 3. 답 저장 + 퀴즈 여러 개

In [16]:
class Quiz(BaseModel):
    """OX 퀴즈의 정보를 추출합니다"""
    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)")
    options: List[str] = Field(..., description="퀴즈의 답은 O 또는 X 입니다.")
    answer: str = Field(..., description="정답은 O 또는 X 입니다.")  # 정답 추가

llm = ChatUpstage(model="solar-pro", temperature=0.8)

# 텍스트를 분석하여 관련 개념에 대한 OX 퀴즈를 생성하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a quiz master who generates OX quizzes based on the provided context. Ensure the quizzes are scientifically accurate and in OX format.",
        ),
        (
            "human",
            "제공된 내용에 대해 OX 퀴즈를 5개 만들어 주세요. 텍스트 내용에서 잘못된 부분이 있으면 그것도 반영하여 출제해주세요. \n\nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format."),
    ]
)

class QuizList(BaseModel):
    quizzes: List[Quiz]

start_time = time.time()

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 주어진 내용에 대해 퀴즈 생성
generated_quizzes = chain.invoke({"topic": "별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작. 중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. 원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승."})

# 생성된 퀴즈 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):
    print(f"{idx}. {quiz.question} (난이도: {quiz.level})")
    for i, opt in enumerate(quiz.options):
        print(f"   {i+1}) {opt}")
    print(f"정답: {quiz.answer}")  # 정답 출력
    print()

end_time=time.time()
print(end_time-start_time)

1. 성운 단계와 원시별 단계에서 별이 형성되는가? (난이도: 보통)
   1) O
   2) X
정답: O

2. 성운 단계와 원시별 단계에서 핵융합 반응이 일어나는가? (난이도: 보통)
   1) O
   2) X
정답: X

3. 성운 단계와 원시별 단계에서 별이 형성되는가? (난이도: 보통)
   1) O
   2) X
정답: X

4. 성운 단계와 원시별 단계에서 중심부 온도가 급격히 상승하는가? (난이도: 보통)
   1) O
   2) X
정답: O

5. 성운 단계와 원시별 단계에서 핵융합 반응이 일어나는가? (난이도: 보통)
   1) O
   2) X
정답: X

13.313021898269653


In [17]:
generated_quizzes

QuizList(quizzes=[Quiz(question='성운 단계와 원시별 단계에서 별이 형성되는가?', level='보통', options=['O', 'X'], answer='O'), Quiz(question='성운 단계와 원시별 단계에서 핵융합 반응이 일어나는가?', level='보통', options=['O', 'X'], answer='X'), Quiz(question='성운 단계와 원시별 단계에서 별이 형성되는가?', level='보통', options=['O', 'X'], answer='X'), Quiz(question='성운 단계와 원시별 단계에서 중심부 온도가 급격히 상승하는가?', level='보통', options=['O', 'X'], answer='O'), Quiz(question='성운 단계와 원시별 단계에서 핵융합 반응이 일어나는가?', level='보통', options=['O', 'X'], answer='X')])

### 4. 해설 저장 & 옵션 제거 & 난이도 제거 & 프롬프트 일부 수정

In [32]:
class Quiz(BaseModel):
    """OX 퀴즈의 정보를 추출합니다"""
    question: str = Field(..., description="퀴즈의 질문")
    answer: str = Field(..., description="정답은 O 또는 X 입니다.")  # 정답
    explanation: str = Field(..., description="문제의 해설을 제공합니다.")  # 해설

class QuizList(BaseModel):
    quizzes: List[Quiz]

llm = ChatUpstage(model="solar-pro", temperature=0.8)

# 퀴즈 생성 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a quiz master who generates OX quizzes based on the provided context. Ensure the quizzes are scientifically accurate and in OX format.",
        ),
        (
            "human",
            "제공된 내용에 대해 OX 퀴즈를 5개 만들어 주세요. "
            "질문은 개념 이해와 문제 해결 능력을 평가할 수 있어야 합니다."
            "정답이 'O'인 문제와 'X'인 문제의 비율은 균형 있게 구성해주세요."
            "질문의 난이도는 수능 수준에 맞춰 구체적이고 사고를 요하는 내용을 포함해야 합니다."
            "각 질문의 정답에 대한 설명은 간결하지만 충분히 납득 가능하게 작성해주세요."
            "각 문제에 대해 정답(O 또는 X)과 정확한 해설도 제공해주세요. \n\nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format."),
    ]
)

start_time = time.time()

llm_with_structured_output = llm.with_structured_output(QuizList)
chain = prompt | llm_with_structured_output

# 주어진 내용에 대해 퀴즈 생성
generated_quizzes = chain.invoke({
    "topic": "별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작. "
             "중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. "
             "원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승."
})

# 생성된 퀴즈 출력
for idx, quiz in enumerate(generated_quizzes.quizzes, 1):
    print(f"{idx}. {quiz.question}")
    print(f"정답: {quiz.answer}")
    print(f"해설: {quiz.explanation}")
    print()

end_time = time.time()
print(end_time - start_time)


1. 성운이 수축하여 차가워지고 밀도가 높아지면, 중심부에 원시별이 형성된다.
정답: O
해설: 성운이 수축하면서 중심부가 차갑고 밀도가 높아지면서 원시별이 형성된다.

2. 원시별 단계에서 중심부는 1000만 K로 가열된다.
정답: X
해설: 원시별 단계에서 중심부는 약 1천만 K로 가열된다.

3. 성운 단계에서 성운은 가스와 먼지로 이루어져 있다.
정답: O
해설: 성운은 가스와 먼지로 이루어져 있다.

4. 원시별 단계에서 중심부 온도는 1000만 K보다 낮다.
정답: X
해설: 원시별 단계에서 중심부 온도는 약 1천만 K로 상승한다.

5. 성운 단계에서 성운은 밀도가 증가한다.
정답: O
해설: 성운 단계에서 성운은 밀도가 증가한다.

14.098474264144897


In [27]:
print(generated_quizzes)

None
